# Brain Tumor Segmentation
- Inverstigation of UNETR architecture
- 26.02.2024
- by gromdimon

## Setup and Imports
First of all you'll need to activate the conda environment and install all the necessary packages. 
Then you can import the libraries and modules that you'll need for the project.

In [3]:
import os
import monai
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader, decollate_batch
from monai.handlers.utils import from_engine
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.networks.nets import SegResNet
from monai.transforms import (
    Activations,
    Activationsd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    Invertd,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    Spacingd,
    EnsureTyped,
    EnsureChannelFirstd,
)
from monai.utils import set_determinism

import torch

print_config()

MONAI version: 1.3.0
Numpy version: 1.26.4
Pytorch version: 2.2.2
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /Users/<username>/Library/Caches/pypoetry/virtualenvs/brain-segment-kVyzAWtz-py3.10/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 10.2.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.8
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: NOT INS

In [4]:
# Add local directories to use them as module
import sys
sys.path.append("../") # go to parent dir
sys.path.append("../..")

In [5]:
# Set deterministic training for reproducibility
set_determinism(seed=0)

## Data Preparation
The dataset is already prepared and can be accessed via DecathlonDataet class from MONAI library.
Utilize the utility functions to load the data.

In [6]:
from src.utils import load_data

# Training data
train_loader, train_ds = load_data(train=True)
# Validation data
val_loader, val_ds = load_data(train=False)

Data directory: /Users/gromdimon/Working/brain-segment/src/data


/Users/gromdimon/Library/Caches/pypoetry/virtualenvs/brain-segment-kVyzAWtz-py3.10/lib/python3.10/site-packages/monai/apps/utils.py:115: UserWarning: tqdm is not installed, will not show the downloading progress bar.
  warnings.warn("tqdm is not installed, will not show the downloading progress bar.")


KeyboardInterrupt: 

## Data Analysis
Get the basic information about the dataset: number of samples, classes, shapes of the images and labels.
Visualize the data to understand the structure and the content of the dataset.

In [ ]:
# Check data shape and visualize
# pick one image from DecathlonDataset to visualize and check the 4 channels
val_data_example = val_ds[2]
print(f"image shape: {val_data_example['image'].shape}")
plt.figure("image", (24, 6))
for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.title(f"image channel {i}")
    plt.imshow(val_data_example["image"][i, :, :, 60].detach().cpu(), cmap="gray")
plt.show()
# also visualize the 3 channels label corresponding to this image
print(f"label shape: {val_data_example['label'].shape}")
plt.figure("label", (18, 6))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.title(f"label channel {i}")
    plt.imshow(val_data_example["label"][i, :, :, 60].detach().cpu())
plt.show()